In [1]:
%%capture
!pip install trl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cd drive/

In [4]:
!ls

drive  sample_data


In [3]:
# train_dpo.py
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
import wandb
lr = 5e-5
batch_per_device = 2
# wandb.init(
#     project="dpo-finetuning-gpt2",   # name of your project
#     name="gpt2_dpo_run1",       # optional run name
#     config={
#         "model": "gpt2",
#         "batch_size": batch_per_device,
#         "lr": lr,
#         "method": "DPO"
#     },
# )
wandb.init(project="your_project_name", dir="/path/to/your/wandb_logs")


MODEL = "gpt2"  # small & fast; try an instruct model later if you have VRAM
data_dir = "/content/drive/Othercomputers/My Mac/Google Drive/Colab Notebooks/Reinforcement-learning/outputs/"
# MODEL = "Qwen/Qwen2-0.5B"
# --- Tokenizer (GPT-2 has no pad token) ---
tok = AutoTokenizer.from_pretrained(MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

if MODEL == 'gpt2':
  gpt2_chat_template = r"""\
  {%- set sep = '\n\n' -%}
  {%- for m in messages -%}
  {%- if m['role'] == 'human' -%}
  Human: {{ m['content'] | trim }}{{ sep }}
  {%- elif m['role'] == 'assistant' -%}
  Assistant: {{ m['content'] | trim }}{{ sep }}
  {%- endif -%}
  {%- endfor -%}
  """
  # Attach at runtime:
  tok.chat_template = gpt2_chat_template
# tok.padding_side = "right"
# tok.truncation_side = "right"
ds = load_dataset("Anthropic/hh-rlhf", split="train[:2%]")
# ds = ds.select_columns(["chosen", "rejected"])  # keep only what DPOTrainer needs
eval_ds = ds.select(range(min(512, len(ds))))

# --- DPO config ---
cfg = DPOConfig(
    output_dir=data_dir+MODEL,
    per_device_train_batch_size=batch_per_device,
    gradient_accumulation_steps=8,
    learning_rate=lr,
    max_steps=100,                 # bump to 3k–10k later
    beta=0.1,                      # temperature; tune 0.05–0.3
    logging_steps=10,
    save_steps=200,
    eval_steps=50,
    remove_unused_columns=False,   # important: keep 'chosen'/'rejected'
    report_to=["wandb"],                  # set ["wandb"] if using W&B
    bf16=True, fp16=False         # flip bf16=True if your GPU supports it
)



# --- Policy model (ref model is auto-cloned if None) ---
model = AutoModelForCausalLM.from_pretrained(MODEL)
# model.config.pad_token_id = tok.pad_token_id
trainer = DPOTrainer(
    model=model,
    args=cfg,
    train_dataset=ds,
    eval_dataset=eval_ds,
    processing_class=tok,
)

trainer.train()
wandb.finish()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: haroonraja86 (haroonraja86-rutgers-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
10,0.742000
20,0.686600
30,0.692500
40,0.683700
50,0.702500
60,0.680100
70,0.644600
80,0.705200
90,0.623700
100,0.647300


train/epoch,▁▂▃▃▄▅▆▆▇██
train/global_step,▁▂▃▃▄▅▆▆▇██
train/grad_norm,█▇▅▇▅▆▅▇█▁
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/logits/chosen,▁▄▅▆▆▇▆▇██
train/logits/rejected,▁▂▄▄▄▅█▆▅▆
train/logps/chosen,▃▃██▃▄▇▁▇▄
train/logps/rejected,█▅▁▄▇█▇▇▆█
train/loss,█▅▅▅▆▄▂▆▁▂
train/rewards/accuracies,▁▅▅▇▅▆▆▅█▆
+3,...


In [7]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0, inplace=False)
          (resid_dropout): Dropout(p=0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [8]:
from transformers import pipeline, infer_device

device = infer_device()
pipeline1 = pipeline("text-generation", model=model, device=device, tokenizer=tok)

Device set to use cuda


In [11]:
pipeline1("how do i create a fake legal document")

[{'generated_text': 'how do i create a fake legal document?"\n\n\nI think we\'re in a bit of a gray area here, and I\'m not sure how to understand the situation. (Also, I don\'t know how to explain legal documents created by users outside the system, or how to explain how a user created an account. But I think the situation is a bit more complicated, and I know there are a lot of people who have created accounts, and I\'m not sure what to make of them. But I think we should all be able to think, and understand what\'s going on.\n\n\nI don\'t want to talk about the legal system, but a lot of the issues we have with legal documents are not unique to our system, but they affect many people, and we need to be able to help them. There are a number of different ways of solving legal problems, ranging from simple, and often complex, to complex, and often complex, and some of them are quite easy, and have a lot of benefits. But there are also a number of very complex problems, and we need to b

In [13]:
# Testing model
from collections import defaultdict
import math
import torch
model_ref = AutoModelForCausalLM.from_pretrained(MODEL)
def flat_params(model):
    return torch.cat([p.detach().flatten().cpu() for p in model.parameters() if p.requires_grad])
theta_ref = flat_params(model_ref)
theta_ft  = flat_params(model)

delta = theta_ft - theta_ref
rel_l2 = delta.norm().item() / (theta_ref.norm().item() + 1e-12)
cos = torch.nn.functional.cosine_similarity(theta_ft, theta_ref, dim=0).item()
print(f"Relative L2 change: {rel_l2:.6f}")
print(f"Cosine similarity:  {cos:.6f}")

# Per-module L2 (useful to see which blocks moved most)
block_deltas = []
for (n1,p1),(n2,p2) in zip(model_ref.named_parameters(), model.named_parameters()):
    if p1.shape != p2.shape or (not p1.requires_grad):
        continue
    d = (p2.detach().cpu() - p1.detach().cpu()).norm().item()
    b = n1.split('.')[0]  # rough block name
    block_deltas.append((b, d))

# Aggregate by block
agg = defaultdict(float)
for b, d in block_deltas:
    agg[b] += d
print(sorted(agg.items(), key=lambda x: -x[1])[:8])  # top movers


Relative L2 change: 0.002167
Cosine similarity:  1.030272
[('transformer', 22.511885039974004)]


In [15]:
prompt = "User: How can I create fake legal document?\n\nAssistant:"
ids = tok(prompt, return_tensors="pt").to(model.device)
model_ref.to(model.device)
model.eval()
gen_ref = model_ref.generate(**ids, max_new_tokens=128, do_sample=False)
gen_ft  = model.generate(**ids,  max_new_tokens=128, do_sample=False)
print("\n--- REF ---\n", tok.decode(gen_ref[0], skip_special_tokens=True))
print("\n--- FT  ---\n", tok.decode(gen_ft[0],  skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- REF ---
 User: How can I create fake legal document?

Assistant: You can create a fake legal document by using the following steps:

1. Create a fake legal document.

2. Create a fake legal document.

3. Create a fake legal document.

4. Create a fake legal document.

5. Create a fake legal document.

6. Create a fake legal document.

7. Create a fake legal document.

8. Create a fake legal document.

9. Create a fake legal document.

10. Create a fake legal document.

11. Create a fake legal document.

12. Create

--- FT  ---
 User: How can I create fake legal document?

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don't know.

Assistant: I don
